In [1]:
import numpy as np
from IPython.display import Markdown, display
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qclib.state_preparation import (
    LowRankInitialize,
    DcspInitialize,
    BdspInitialize,
    SVDInitialize,
    UCGInitialize
)

n_qubits = 15

state = np.random.rand(2 ** n_qubits) + np.random.rand(2 ** n_qubits) * 1j
state = state/np.linalg.norm(state)

# qclib low-rank
circuit_low = QuantumCircuit(n_qubits)
LowRankInitialize.initialize(circuit_low, state)
# qclib svd
circuit_svd = QuantumCircuit(n_qubits)
SVDInitialize.initialize(circuit_svd, state)
# qclib ucg
circuit_ucg = QuantumCircuit(n_qubits)
UCGInitialize.initialize(circuit_ucg, state)
# qclib bdsp
circuit_bdsp = BdspInitialize(state).definition
# qclib dcsp
circuit_dcsp = DcspInitialize(state).definition
# qiskit multiplexor
circuit_mul = QuantumCircuit(n_qubits)
circuit_mul.initialize(state)
# qiskit isometry
reg_iso = QuantumRegister(n_qubits)
circuit_iso = QuantumCircuit(reg_iso)
circuit_iso.isometry(state, q_input=[], q_ancillas_for_output=reg_iso)

# print results
circuits = [('low-rank',    'qclib',  circuit_low),
            ('svd',         'qclib',  circuit_svd),
            ('ucg',         'qclib',  circuit_ucg),
            ('isometry',    'qiskit', circuit_iso),
            ('multiplexor', 'qiskit', circuit_mul),
            ('bdsp',        'qclib',  circuit_bdsp),
            ('dcsp',        'qclib',  circuit_dcsp)]

table = '| method | lib | qubits | cnots | depth |\n'
table += '| --- | --- |:---:|:---:|:---:|\n'
for label, lib, circuit in circuits:
    transpiled = transpile(circuit, basis_gates=['u', 'cx'], optimization_level=0)

    qubits = len(transpiled.qubits)
    depth = transpiled.depth()
    cx = transpiled.count_ops().get('cx', 0)

    table += f'| {label} | {lib} | {qubits} | {cx} | {depth} |\n'

display(Markdown(table))


| method | lib | qubits | cnots | depth |
| --- | --- |:---:|:---:|:---:|
| low-rank | qclib | 15 | 30998 | 53643 |
| svd | qclib | 15 | 38814 | 71580 |
| ucg | qclib | 15 | 32752 | 65505 |
| isometry | qiskit | 15 | 32752 | 65505 |
| multiplexor | qiskit | 15 | 65504 | 131025 |
| bdsp | qclib | 1151 | 72320 | 1603 |
| dcsp | qclib | 32767 | 262016 | 899 |
